# Task 2: Price a commodity storage contract

The desk now has the price data they need. The final ingredient before they can begin trading with the client is the pricing model. Alex tells you the client wants to start trading as soon as possible. They believe the winter will be colder than expected, so they want to buy gas now to store and sell in winter in order to take advantage of the resulting increase in gas prices. They ask you to write a script that they can use to price the contract. Once the desk are happy, you will work with engineering, risk, and model validation to incorporate this model into production code.

## Provided Solution

#### **Change of Structure:**
- Transitioned to an object-oriented style of programming to ensure modularity, separation of concerns, and clearer code structure.
- Distinctly separated the functionalities: forecasting from data and pricing the contract.
- A test case has been implemented to validate the entire flow.

#### **Forecast Code:**
- Used the ARIMA model for forecasting the gas prices.
- Split the data into training and testing sets based on a predefined cut-off. The training set is used to train the ARIMA model, and the test set is used for validation.
- Utilised the pmdarima package to automatically find the best ARIMA parameters for the training set.

#### **Pricing Engine Code:**
- Created a Contract class to capture the details of the gas contract, including injection and withdrawal dates, rates, storage costs, and maximum storage volume.
- Developed a PricingEngine that utilises the forecast model to price the contract.
- For better accuracy, linear interpolation is performed on the data to ensure that prices are available for every day.
- Costs are computed based on the injection and withdrawal dates provided in the contract. The buying cost (injection) is subtracted, and the selling revenue (withdrawal) is added.
- Additionally, storage costs are calculated based on the amount of gas stored.

#### **Test Case Code:**
- Gas was procured during the first two weeks of July and intended to be sold in the first two weeks of February.
- Leveraged the DataImporter to fetch data from the CSV file.
- Initialised the ARIMA model and performed the analysis.
- Defined the contract details, including specified injection and withdrawal dates and rates.
- Employed the PricingEngine to calculate the contract's price, showcasing the end-to-end functionality of the solution.





### Forecast Code

In [2]:
from abc import ABC, abstractmethod
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
import matplotlib.pyplot as plt


# Module-level constants
TRAINING_CUT_OFF = -12
ARIMA_PARAMS = {
    'start_p': 0, 'start_q': 0,
    'max_p': 3, 'max_q': 3, 'm': 12,
    'seasonal': True,
    'trace': True, 
    'error_action': 'ignore',
    'suppress_warnings': True,
    'stepwise': True
}


class DataImporter:
    """Handles data importing."""

    @staticmethod
    def import_csv(file_name: str) -> pd.DataFrame:
        df = pd.read_csv(file_name, parse_dates=['Dates'])
        df.set_index('Dates', inplace=True)
        return df


class ForecastModel(ABC):
    """Abstract base for pricing models."""

    def __init__(self, data: pd.DataFrame):
        self.data = data

    @abstractmethod
    def analysis(self):
        pass

    @abstractmethod
    def plot_graph(self):
        pass


class ARIMAModel(ForecastModel):
    """Implementation of ARIMA Pricing Model."""
    
    def __init__(self, data: pd.DataFrame):
        self.train = data.iloc[:TRAINING_CUT_OFF]
        self.test = data.iloc[TRAINING_CUT_OFF:]
        self.forecast = None
        super().__init__(data)

    def analysis(self):
        stepwise_fit = auto_arima(self.train, **ARIMA_PARAMS)

        model = ARIMA(self.train, order=stepwise_fit.order, seasonal_order=stepwise_fit.seasonal_order)
        fit_model = model.fit()

        self.forecast = fit_model.get_forecast(steps=len(self.test)).predicted_mean

    def plot_graph(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.train.index, self.train, label="Training")
        plt.plot(self.test.index, self.test, label="Validation")
        plt.plot(self.test.index, self.forecast, label="Forecast", color='red')
        plt.legend(loc="upper left")
        plt.title("Natural Gas Prices Forecasting with ARIMA")
        plt.grid()
        plt.show()

### Pricing Engine Code

In [5]:
from typing import List, Tuple
import pandas as pd


class Contract:
    """Represents the gas contract."""
    def __init__(self, 
                 injection_dates: List[pd.Timestamp], 
                 withdrawal_dates: List[pd.Timestamp],
                 injection_rate: float,
                 withdrawal_rate: float,
                 max_volume: float,
                 storage_cost: float):
        
        self.injection_dates = injection_dates
        self.withdrawal_dates = withdrawal_dates
        self.injection_rate = injection_rate
        self.withdrawal_rate = withdrawal_rate
        self.max_volume = max_volume
        self.storage_cost = storage_cost
        self.stored_gas = 0


class PricingEngine:
    """Calculates the price of the gas contract."""

    def __init__(self, model: ForecastModel):
        self.model = model

    def price_contract(self, contract: Contract) -> float:
        cost = 0
        # Assuming data contains the pricing data for the required dates
        data = self.model.data

        # Linear Interpolation so price is available for every day
        data = data.resample('D').interpolate(method='linear')

        # Handle Injection - can handle multiple injection dates
        for date in contract.injection_dates:
            if date in data.index:
                cost -= contract.injection_rate * data.loc[date].item()

        # Handle Withdrawal - can handle multiple withdrawal dates
        for date in contract.withdrawal_dates:
            if date in data.index:
                cost += contract.withdrawal_rate * data.loc[date].item()

        # Handle Storage Cost
        cost += contract.stored_gas * contract.storage_cost

        return cost

### Test Case

In [6]:
import pandas as pd

if __name__ == "__main__":
    # Get data
    file_name = 'Nat_Gas.csv'
    data = DataImporter.import_csv(file_name)

    # Initialize and analyze the ARIMA model
    model = ARIMAModel(data)
    model.analysis()
    
    # Define the contract

    # Buy in the summer and sell in the winter
    injection_dates = [pd.Timestamp('2023-07-01'), pd.Timestamp('2023-07-12')]
    withdrawal_dates = [pd.Timestamp('2024-02-01'), pd.Timestamp('2024-02-12')]
    # Rate at which gas can be injected or withdrawn
    injection_rate = 100
    withdrawal_rate = 100
    # Maximum volume of gas that can be stored
    max_volume = 1000
    storage_cost = 0.05
    contract = Contract(injection_dates, withdrawal_dates, injection_rate, withdrawal_rate, max_volume, storage_cost)

    # Price the contract using the pricing engine
    pricing_engine = PricingEngine(model)
    price = pricing_engine.price_contract(contract)

    print(f"Contract Price: ${price:.2f}")

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[12] intercept   : AIC=35.472, Time=0.16 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=37.411, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=33.531, Time=0.11 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=34.978, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=35.777, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=39.410, Time=0.01 sec
 ARIMA(1,1,0)(2,0,0)[12] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(1,1,0)(1,0,1)[12] intercept   : AIC=34.343, Time=0.44 sec
 ARIMA(1,1,0)(0,0,1)[12] intercept   : AIC=35.024, Time=0.07 sec
 ARIMA(1,1,0)(2,0,1)[12] intercept   : AIC=34.899, Time=0.70 sec
 ARIMA(0,1,0)(1,0,0)[12] intercept   : AIC=33.476, Time=0.07 sec
 ARIMA(0,1,0)(2,0,0)[12] intercept   : AIC=35.472, Time=0.21 sec
 ARIMA(0,1,0)(0,0,1)[12] intercept   : AIC=33.882, Time=0.09 sec
 ARIMA(0,1,0)(2,0,1)[12] intercept   : AIC=37.472, Time=0.10 sec
 ARIMA(0,1,1)(1,0,0)[12] intercept   : AIC=33.364,

/Users/patrickmontgomery/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/patrickmontgomery/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/patrickmontgomery/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
